In [ ]:
import getpass
import os
import pandas as pd
import json
from transformers import pipeline

In [ ]:
file_path = "/content/drive/MyDrive/FINKI/WBS, IKT KG Project/data/sentences/sentences.json"

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/FINKI/WBS, IKT KG Project/data/patients_filtered.csv", index_col="patient_id").reset_index()

In [ ]:
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-3B-Instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
example_report = """A 41-year-old man presented to the emergency department following an automotive accident where he was thrown from his motorcycle traveling approximately 35 mph. The patient denied loss-of-consciousness and chiefly complained of left wrist pain at presentation with exam demonstrating tenderness and swelling. Radiographs () revealed a volar dislocation and rotatory deformity of the proximal pole of the scaphoid. """

In [ ]:
example_sentences = """41-year-old man
automotive accident
left wrist pain
volar dislocation of scaphoid
rotatory deformity of scaphoid
closed reduction failure
operative management
dorsal approach to radiocarpal joint"""

In [ ]:
system_message = f"""You are assistant for extracting information from medical reports."""

In [ ]:
def generate_sentences(report):
  human_message = f"""Your job is to extract information in simple and short sentences from this report.
The information needs to be describing something about the patient.
Only respond with the short and simple object descriptions seperated only by a new line.
Give at least 10 sentences. Respond only with the sentences.

Example:
Report:
{example_report}

Sentences:
{example_sentences}

Begin!
Report:
{report}
"""

  messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": human_message}
  ]
  result = pipe(messages, do_sample=False)
  return result[0]["generated_text"][2]["content"]

In [ ]:
def save_object(obj):
  try:
    with open(file_path, "r") as f:
      data = json.load(f)
  except FileNotFoundError:
      data = []

  data.append(obj)
  with open(file_path, "w") as f:
    json.dump(data, f, indent=4)

In [ ]:
starting_index = 3380
for index, row in df.iterrows():
  if index > starting_index:
    report = row["patient"]
    patient_id = row["patient_uid"]
    age = row["age"]
    gender = row["gender"]
    sentences = generate_sentences(report).split("\n")
    obj = {
        "report": report,
        "patient_id": patient_id,
        "age": age,
        "gender": gender,
        "sentences": sentences
    }
    save_object(obj)
    print("*" * 20)
    print(obj)
    print(f"Finished report {index}")
    print("*" * 20)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


********************
{'report': 'Male, 68 years old, admitted to hospital on March 18, 2013, pathologically diagnosed as “Left soft palate squamous cell carcinoma”. Four days after admission, “Extended tumor resection combined with cervical lymph node clearance + left anterolateral femoral myocutaneous flap transfer repair + tracheotomy” was performed under general anesthesia. On the 4th day after the operation, the patient developed active bleeding in the mouth and nose, and dark red liquid could be drawn from the gastric tube. The patient died after 2 h’ rescue. The causes of death were: 1. Respiratory obstruction. 2. Hemorrhagic shock.', 'patient_id': '7941918-1', 'age': "[[68.0, 'year']]", 'gender': 'M', 'sentences': ['Male, 68 years old', 'Left soft palate squamous cell carcinoma', 'Extended tumor resection', 'Cervical lymph node clearance', 'Left anterolateral femoral myocutaneous flap transfer', 'Tracheotomy', 'Active bleeding in the mouth', 'Dark red liquid from the gastric tub

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


********************
{'report': 'Male, 63 years old, admitted to hospital on December 9, 2013, diagnosed as “Left soft palate squamous cell carcinoma”, had a history of hypertension for more than 10 years. Four days after admission, “Extended tumor resection + left anterolateral femoral myocutaneous flap transfer repair + tracheotomy” was performed under general anesthesia. On the 2nd day after the operation, the patient developed “loss of consciousness, cyanosis of the lips, no spontaneous breathing, no heartbeat and disappearance of carotid artery beats”, and died 1.5 h’ after rescue. Cause of death: Cardiac respiratory arrest.', 'patient_id': '7941918-2', 'age': "[[63.0, 'year']]", 'gender': 'M', 'sentences': ['Male, 63 years old', 'Left soft palate squamous cell carcinoma', 'Hypertension', 'Extended tumor resection', 'Left anterolateral femoral myocutaneous flap transfer', 'Tracheotomy', 'Loss of consciousness', 'Cyanosis of the lips', 'No spontaneous breathing', 'No heartbeat', 'C

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


********************
{'report': 'A 30-year-old woman with the onset of fever, myalgia and left armpit pain ipsilateral to vaccine injection after the first dose underwent US examination 5 days later. Multiple axillary lymphadenopathies (maximum diameter 15 × 10 mm) with round to oval shape, poor hilar representation, thickened cortex and increased vascularization at color-Doppler were found. No pathological lymph nodes were found in ipsilateral supra and subclavicular lymph node stations as well as in lymph nodes contralateral to the site of vaccine injection.\\nUS follow-up performed 4 weeks later showed stable lymph node size with restored fatty hilum as well as reduced cortical thickness and vascularization (Fig. ).', 'patient_id': '8211958-6', 'age': "[[30.0, 'year']]", 'gender': 'F', 'sentences': ['30-year-old woman', 'fever', 'myalgia', 'left armpit pain', 'axillary lymphadenopathies', 'round lymph nodes', 'thickened lymph node cortex', 'increased vascularization', 'vaccine injec

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


********************
{'report': "A 65-year-old man with a past medical history (PMH) of asthma, hypertension (HTN), hyperlipidemia (HLD), insulin resistance, and obstructive sleep apnea (OSA) was admitted to the hospital due to dyspnea, tachypnea [respiratory rate (RR), 22/min], cough, fever (103°F), and hypoxia [blood oxygen saturation levels (SpO2), 81%] after having a 2-week period of fever, chills, and body aches. Symptoms had worsened in the last 72 h before admission. On admission, the patient was placed on a non-rebreather mask (NRB) at 11 L/min (LPM) due to respiratory distress, which partially improved hypoxia from SpO2 81 to 91%. Three days after admission, the respiratory status worsened revealing diffuse bilateral ground-glass opacification on chest CT scan, along with increased counts of neutrophils in plasma, lymphopenia, and transaminitis. Labs showed increased ferritin and C-reactive protein (CRP) levels (), features compatible with the cytokine release-like syndrome as

KeyboardInterrupt: 